In [1]:
import numpy as np
import pandas as pd

In [2]:
train_new=pd.read_csv(r'C:\Users\Dell\Downloads\agriculture av\train_new.csv')
test_new=pd.read_csv(r'C:\Users\Dell\Downloads\agriculture av\test_new.csv')
samp_sub=pd.read_csv(r'C:\Users\Dell\Downloads\agriculture av\sample_submission.csv')

In [3]:
train_new.head()

,ID,Estimated_Insects_Count,Crop_Type,Soil_Type,Pesticide_Use_Category,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Season,Crop_Damage
0,F00000001,188.0,1,0,1,0.0,0.0,0.0,1,0
1,F00000003,209.0,1,0,1,0.0,0.0,0.0,2,1
2,F00000004,257.0,1,0,1,0.0,0.0,0.0,2,1
3,F00000005,257.0,1,1,1,0.0,0.0,0.0,2,1
4,F00000006,342.0,1,0,1,0.0,0.0,0.0,2,1


In [4]:
#crop,soil,no_doses,season
train_new2=train_new.drop(['ID','Soil_Type','Season'],axis=1)
train_new2.head()

,Estimated_Insects_Count,Crop_Type,Pesticide_Use_Category,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Crop_Damage
0,188.0,1,1,0.0,0.0,0.0,0
1,209.0,1,1,0.0,0.0,0.0,1
2,257.0,1,1,0.0,0.0,0.0,1
3,257.0,1,1,0.0,0.0,0.0,1
4,342.0,1,1,0.0,0.0,0.0,1


In [5]:
from sklearn.model_selection import train_test_split 

X = train_new2.drop('Crop_Damage',axis=1)
y = train_new2['Crop_Damage']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=42)

In [6]:
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
params1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)}

params2={'gamma':[i/10.0 for i in range(0,5)],
 'subsample':[i/10.0 for i in range(6,10)]}

params3={'colsample_bytree':[i/10.0 for i in range(6,10)],
 'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05]}



In [ ]:
model1 = XGBClassifier()

kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=10)
grid_search = RandomizedSearchCV(model1, params1, scoring="accuracy", n_iter = 500, cv=kfold)
grid_result = grid_search.fit(X,y)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

In [7]:
rf = XGBClassifier(max_depth = 5,
                n_estimators=1000,
                learning_rate=0.1, 
                min_child_weight = 5,
                seed=1301)
                
rf.fit(X_train, y_train, early_stopping_rounds=50, eval_metric="merror", eval_set=[(X_train, y_train), (X_test, y_test)])

[0]	validation_0-merror:0.15754	validation_1-merror:0.15946
Multiple eval metrics have been passed: 'validation_1-merror' will be used for early stopping.

Will train until validation_1-merror hasn't improved in 50 rounds.
[1]	validation_0-merror:0.15754	validation_1-merror:0.15946
[2]	validation_0-merror:0.15754	validation_1-merror:0.15946
[3]	validation_0-merror:0.15591	validation_1-merror:0.15748
[4]	validation_0-merror:0.15583	validation_1-merror:0.15731
[5]	validation_0-merror:0.15586	validation_1-merror:0.15738
[6]	validation_0-merror:0.15586	validation_1-merror:0.15738
[7]	validation_0-merror:0.15583	validation_1-merror:0.15731
[8]	validation_0-merror:0.15583	validation_1-merror:0.15731
[9]	validation_0-merror:0.15589	validation_1-merror:0.15731
[10]	validation_0-merror:0.15583	validation_1-merror:0.15731
[11]	validation_0-merror:0.15583	validation_1-merror:0.15731
[12]	validation_0-merror:0.15583	validation_1-merror:0.15731
[13]	validation_0-merror:0.15583	validation_1-merror:0

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=5, missing=nan, monotone_constraints='()',
              n_estimators=1000, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=1301, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, seed=1301, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [8]:
from sklearn.metrics import accuracy_score
y_pred = rf.predict(X_test)

print(accuracy_score(y_test, y_pred))

0.8449734006274724


In [9]:
test_new2=test_new.drop(['ID','Soil_Type','Season'],axis=1)
test_id=test_new['ID']
pred_test=rf.predict(test_new2)
pred_test

array([1, 1, 1, ..., 0, 0, 0], dtype=int64)

In [10]:
d={'ID':test_id,'Crop_Damage':pred_test}
sub=pd.DataFrame(data=d)
sub

,ID,Crop_Damage
0,F00000002,1
1,F00000007,1
2,F00000011,1
3,F00000013,1
4,F00000014,1
...,...,...
59305,F00155937,0
59306,F00155940,0
59307,F00155941,0
59308,F00155943,0


In [11]:
sub.to_csv(r'C:\Users\Dell\Downloads\agriculture av\submission5.csv',index=False)

In [ ]:
train_new2.corr()

In [ ]:
#crop,soil,no_doses,season